In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Chromophobe/GSE19949'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrative genome-wide expression profiling identifies three distinct molecular subgroups of renal cell carcinoma with different patient outcome"
!Series_summary	"Background: Renal cell carcinoma (RCC) is characterized by a number of diverse molecular aberrations that differ among individuals. Recent approaches to molecularly classify RCC were based on clinical, pathological as well as on single molecular parameters. As a consequence, gene expression patterns reflecting the sum of genetic aberrations in individual tumors may not have been recognized. In an attempt to uncover such molecular features in RCC, we used a novel, unbiased and integrative approach."
!Series_summary	"Methods: We integrated gene expression data from 97 primary RCCs of different pathologic parameters, 15 RCC metastases as well as 34 cancer cell lines for two-way nonsupervised hierarchical clustering using gene groups suggested by the PANTHER Classification System. We depict

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 7
age_row = None
gender_row = 6

def convert_trait(value):
    if 'chromophobe' in value.lower():
        return 1
    else:
        return 0

def convert_age(value):
    try:
        age = int(value.split(':')[1].strip())
        return age
    except ValueError:
        return None

def convert_gender(value):
    gender_value = value.split(':')[1].strip().lower()
    if gender_value == 'male':
        return 1
    elif gender_value == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE19949', './preprocessed/Kidney_Chromophobe/cohort_info.json', is_gene_available, trait_row is not None)

selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Chromophobe', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Kidney_Chromophobe/trait_data/GSE19949.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM498450': [0, 1], 'GSM498451': [0, None], 'GSM498452': [0, None], 'GSM498453': [0, 1], 'GSM498454': [0, None], 'GSM498455': [0, None], 'GSM498456': [0, None], 'GSM498457': [0, 0], 'GSM498458': [0, None], 'GSM498459': [0, None], 'GSM498460': [0, 1], 'GSM498461': [0, 1], 'GSM498462': [0, None], 'GSM498463': [0, 1], 'GSM498464': [0, 1], 'GSM498465': [0, None], 'GSM498466': [0, None], 'GSM498467': [0, None], 'GSM498468': [0, None], 'GSM498469': [0, None], 'GSM498470': [0, None], 'GSM498471': [0, None], 'GSM498472': [0, 0], 'GSM498473': [0, None], 'GSM498474': [0, 0], 'GSM498475': [0, 0], 'GSM498476': [0, None], 'GSM498477': [0, 1], 'GSM498478': [0, 1], 'GSM498479': [0, 1], 'GSM498480': [0, 1], 'GSM498481': [1, None], 'GSM498482': [0, None], 'GSM498483': [0, None], 'GSM498484': [0, 1], 'GSM498485': [0, 1], 'GSM498486': [0, 1], 'GSM498487': [0, None], 'GSM498488': [0, None], 'GSM498489': [0, 1], 'GSM498490': [0, 0], 'GSM498491': [0, 1], 'GSM498492': [0, None], 'GSM498493': [0, None], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# identifier_key and gene_symbol_key based on the dictionary from STEP5
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens Pax8 mRNA', 'L36861 /FEATURE=expanded_cds /DEFINITION=HUMGCAPB Homo sapiens 

### Step 6: Gene Identifier Mapping

In [7]:
# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert probe IDs to gene symbols and get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Chromophobe/gene_data/GSE19949.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Chromophobe')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE19949', './preprocessed/Kidney_Chromophobe/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Chromophobe/GSE19949.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Kidney_Chromophobe', the least common label is '1.0' with 1 occurrences. This represents 2.33% of the dataset.
The distribution of the feature 'Kidney_Chromophobe' in this dataset is severely biased.

For the feature 'Gender', the least common label is '0.0' with 11 occurrences. This represents 25.58% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

